In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import os, warnings # 경고메세지 라이브러리
import re   # 정규표현식
import time
warnings.filterwarnings(action='ignore') # 경고메세지 무시

import pandas as pd

In [7]:
# 수집된 데이터 가져오기
dat = pd.read_excel('./lotteria_all_store.xlsx')
# dat = dat.iloc[0:320]
# dat = dat.iloc[320:640]
dat = dat.iloc[640:960]
# dat = dat.iloc[960:]
dat

,지점명,주소,주소1,주소2,주소3
640,망우,서울특별시 중랑구 봉우재로 239,서울특별시,중랑구,봉우재로 239
641,울산세이브존,울산광역시 남구 번영로 250번길 9,울산광역시,남구,번영로 250번길 9
642,NC강서,서울특별시 강서구 강서로 56길 17,서울특별시,강서구,강서로 56길 17
643,영광,전라남도 영광군 영광읍신남로 185-2,전라남도,영광군,영광읍신남로 185-2
644,광주동강대,광주광역시 북구 동문대로 69,광주광역시,북구,동문대로 69
...,...,...,...,...,...
955,롯데마트서대전,대전광역시 유성구 유성대로 26-37롯데마트(서대전점) 2층 롯데리아,대전광역시,유성구,유성대로 26-37롯데마트(서대전점) 2층 롯데리아
956,NC청주,충청북도 청주시 흥덕구 2순환로 1233(가경동),충청북도,청주시,흥덕구 2순환로 1233(가경동)
957,제천청전,충청북도 제천시 청전동 청전대로169,충청북도,제천시,청전동 청전대로169
958,대전역사,대전광역시 동구 중앙로 215(정동),대전광역시,동구,중앙로 215(정동)


In [8]:
addr = {'서울시':'서울특별시',
        '서울':'서울특별시',
        '부산시':'부산광역시',
        '부산':'부산광역시',
        '대구':'대구광역시',
        '인천':'인천광역시',
        '광주':'광주광역시',
        '대전시':'대전광역시',
        '대전':'대전광역시',
        '울산':'울산광역시',
        '세종':'세종특별자치시',
        '경기':'경기도',
        '강원':'강원도',
        '충북':'충청북도',
        '충남':'충청남도',
        '전북':'전라북도',
        '전남':'전라남도',
        '경북':'경상북도',
        '경남':'경상남도',
        '제주':'제주특별자치도'}

In [9]:
def change_addr(x) :
    
    s_name_ori = x['지점명']
    
    # D/T 같이 '/'이 포함된 경우 네이버에 제대로 검색되지 않음
    s_name = s_name_ori
    if '/' in s_name :
        s_name = s_name.replace('/', '')
        
    f_name = f'롯데리아 {s_name}'
    
    url = f'https://search.naver.com/search.naver?query={f_name}'
          
    driver.get(url)
    time.sleep(1)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    # 네이버에 '롯데리아 지점명' 검색하여 주소 획득
    try :

        # case1 : 롯데리아 감일
        full_addr = soup.select_one('span.addr').text
        addr1, addr2, addr3 = full_addr.split(' ', 2)
        print(f'{s_name} : {addr1} {addr2} {addr3}')
        x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

    except Exception as e :

        print(f'{s_name} : 1. {e}')
        full_addr = ''

        try :
            store_list = soup.select('div._3ZU00')
            
            # case2 : 롯데리아 의정부화룡역
            if len(store_list) == 0 :
                full_addr = soup.select('div._1mAZf')[1].text

                full_addr = full_addr[:full_addr.rindex('지번')]
                addr1, addr2, addr3 = full_addr.split(' ', 2)
                print(f'{s_name} : {addr1} {addr2} {addr3}')
                x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3

            # 검색목록이 여러개인 경우, 지점명과 일치하는 주소를 획득
            for idx, i in enumerate(store_list) :
                
                except_text = ['전북', '전남', '경북', '경남']
                
                # try 구문을 사용해도 되지만 이미 이중으로 try 선언이 되어 굳이 String으로 해결
                # 지점명에 도명이 포함된 경우, 네이버 검색 시 도명이 없이 롯데리아 진안점으로 검색됨
                # 도명을 제거하여 네이버 검색결과 중 일치하는 지점의 주소를 입력
                s_name2 = 'None 선언시 에러가 생기므로 String으로 None 선언'
                if s_name[:2] in except_text :
                    s_name2 = s_name[2:]

                # case3 : 롯데리아 전북진안, 롯데리아 강남역
                if s_name in i.a.text or s_name2 in i.a.text :
                
                    try :
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[2]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)
                    except :
                        # case4 : 롯데리아 경남함안
                        xpath = f'//*[@id="loc-main-section-root"]/section/div/div[2]/ul/li[{idx+1}]/div[1]/div[2]/div/span/a/span[2]'
                        down = driver.find_element_by_xpath(xpath)

                    down.click()
                    time.sleep(1)
                    
                    # 주소에 도로명과, 복사를 뺴고 입력
                    full_addr = driver.find_element_by_css_selector('div._2b9ic').text[3:-2]
                    addr1, addr2, addr3 = full_addr.strip().split(' ', 2)
                    print(f'{s_name} : {addr1} {addr2} {addr3}')
                    x[ ['주소1', '주소2', '주소3'] ] = addr1, addr2, addr3
                    
                    break
                    
        except Exception as e:

            print(f'{s_name} : 2. {e}')
            
    if len(full_addr) < 1 :
        # drop 처리하려 했으나 작업 편의성을 위해 None 처리
#         print(f'{s_name_ori} : 삭제')
#         dat.drop( dat.loc[ dat['지점명'] == s_name_ori ].index, axis=0, inplace=True)

        print(f'{s_name_ori} : None처리')
        pass
    else :
        return x

# 웹드라이버 실행
driver = webdriver.Chrome('../chromedriver_90')

# 주소1의 이상값을 정상값으로 변경하고 없는 지점은 None처리
dat = dat.apply(lambda x : change_addr(x), axis=1)
# dat = dat.apply(lambda x : change_addr(x) if x['주소1'] not in addr.values() else x, axis=1)

# 웹드라이버 종료
driver.close()

망우 : 1. 'NoneType' object has no attribute 'text'
망우 : 서울특별시 중랑구 봉우재로 239 천우빌딩
울산세이브존 : 울산 남구 번영로250번길 9
NC강서 : 서울 강서구 강서로56길 17
영광 : 전남 영광군 영광읍 신남로 185-2
광주동강대 : 광주 북구 동문대로 69
상계 : 1. 'NoneType' object has no attribute 'text'
상계 : 서울특별시 노원구 한글비석로 369 코끼리약국
대구동호DT : 대구 동구 안심로 288
청양 : 1. 'NoneType' object has no attribute 'text'
청양 : 충청남도 청양군 청양읍 중앙로 147
대전와동DT : 1. 'NoneType' object has no attribute 'text'
대전와동DT : 2. list index out of range
대전와동D/T : None처리
부산미남역 : 부산 동래구 아시아드대로 234
오이도DI : 경기 시흥시 정왕신길로49번길 53
일산킨텍스 : 1. 'NoneType' object has no attribute 'text'
일산킨텍스 : 경기도 고양시 일산서구 킨텍스로 217-59
인천청라 : 1. 'NoneType' object has no attribute 'text'
인천청라 : 인천광역시 서구 중봉대로 587
신경주역사 : 경북 경주시 건천읍 신경주역로 80
부산명지 : 1. 'NoneType' object has no attribute 'text'
부산명지 : 부산광역시 강서구 명지오션시티11로 74
상암 : 1. 'NoneType' object has no attribute 'text'
상암 : 서울특별시 마포구 월드컵북로 402 KGIT 센터
방학 : 1. 'NoneType' object has no attribute 'text'
방학 : 서울특별시 도봉구 방학로 173
시흥월곶 : 1. 'NoneType' object has no attribute 'text'
시

구미진평 : 경상북도 구미시 인동36길 5
양산중부 : 경남 양산시 양산역로 69
포항두호 : 경북 포항시 북구 두호로 26
대구칠곡구암 : 대구 북구 학정로 440
마산호계 : 경남 창원시 마산회원구 내서읍 호원로 220
대구퀸스로드 : 대구 서구 평리로35길 18-8
경남합천 : 1. 'NoneType' object has no attribute 'text'
경남합천 : 경상남도 합천군 합천읍 동서로 85-1
경상대학 : 1. 'NoneType' object has no attribute 'text'
경상대학 : 경상남도 진주시 진주대로 594
울산화정 : 1. 'NoneType' object has no attribute 'text'
울산화정 : 울산광역시 동구 대학길 65
대구방촌 : 대구 동구 화랑로 409
울산호계 : 1. 'NoneType' object has no attribute 'text'
울산호계 : 울산광역시 북구 호계로 302
대구다사 : 1. 'NoneType' object has no attribute 'text'
대구다사 : 대구광역시 달성군 다사읍 달구벌대로 874 A동 107호
경주안강 : 경북 경주시 안강읍 비화원로 77
대구나이스수성 : 대구 수성구 수성로 227
대구논공 : 대구 달성군 논공읍 논공로 741
대구안지랑 : 대구 남구 대명로 160-2
경남창녕 : 1. 'NoneType' object has no attribute 'text'
경남창녕 : 경상남도 창녕군 창녕읍 명덕로 70 롯데리아창녕점
김해진영 : 1. 'NoneType' object has no attribute 'text'
김해진영 : 경상남도 김해시 진영읍 여래로 18 롯데리아 김해진영점
구미오태DT : 경북 구미시 금오대로 292
울산온산 : 1. 'NoneType' object has no attribute 'text'
울산온산 : 울산광역시 울주군 온산읍 덕남로 5
울산태화 : 울산 중구 태화로 174
창원도계 : 경남 창원시 의창구 도계로 99

롯데마트서산 : 충남 서산시 충의로 27
아산둔포 : 1. 'NoneType' object has no attribute 'text'
아산둔포 : 충청남도 아산시 둔포면 둔포중앙로 148
롯데마트당진 : 충남 당진시 정안로 20
홈플러스오창 : 충북 청주시 청원구 오창읍 중심상업1로 8-11
송악 : 1. 'NoneType' object has no attribute 'text'
송악 : 충청남도 당진시 송악읍 신복운로7길 4-4 보경인테리어
홈플러스천안 : 1. 'NoneType' object has no attribute 'text'
홈플러스천안 : 충청남도 천안시 동남구 천안대로 574
충주터미널 : 충북 충주시 봉계1길 49
홈플러스조치원 : 세종 조치원읍 허만석로 60
롯데마트대덕테크노밸리 : 1. 'NoneType' object has no attribute 'text'
롯데마트대덕테크노밸리 : 대전광역시 유성구 테크노중앙로 14 롯데마트대덕테크노밸리점
천안직산 : 1. 'NoneType' object has no attribute 'text'
천안직산 : 충청남도 천안시 서북구 직산읍 봉주로 38 인터넷&게임사이버장
청주산남 : 1. 'NoneType' object has no attribute 'text'
청주산남 : 충청북도 청주시 서원구 산남로70번길 3 1층
대전가오 : 1. 'NoneType' object has no attribute 'text'
대전가오 : 대전광역시 동구 은어송로 60 지티엠빌딩
대전전민 : 대전 유성구 유성대로 1746-1
홈플러스천안신방 : 1. 'NoneType' object has no attribute 'text'
홈플러스천안신방 : 충청남도 천안시 동남구 신촌1길 23 홈에버
조치원 : 1. 'NoneType' object has no attribute 'text'
조치원 : 세종특별자치시 조치원읍 조치원로 27-1 롯데리아조치원점
아산배방 : 1. 'NoneType' object has no attribu

In [10]:
# 주소1의 줄임말을 풀어서 치환하고 None처리 된 행을 제거
dat['주소1'] = dat['주소1'].replace(addr)
dat.dropna(inplace=True)
dat.head()

,지점명,주소,주소1,주소2,주소3
640,망우,서울특별시 중랑구 봉우재로 239,서울특별시,중랑구,봉우재로 239 천우빌딩
641,울산세이브존,울산광역시 남구 번영로 250번길 9,울산광역시,남구,번영로250번길 9
642,NC강서,서울특별시 강서구 강서로 56길 17,서울특별시,강서구,강서로56길 17
643,영광,전라남도 영광군 영광읍신남로 185-2,전라남도,영광군,영광읍 신남로 185-2
644,광주동강대,광주광역시 북구 동문대로 69,광주광역시,북구,동문대로 69


In [11]:
# null 확인
dat.isnull().sum()

지점명    0
주소     0
주소1    0
주소2    0
주소3    0
dtype: int64

In [12]:
# 주소1 이상치 확인
dat['주소1'].unique()

array(['서울특별시', '울산광역시', '전라남도', '광주광역시', '대구광역시', '충청남도', '부산광역시', '경기도',
       '인천광역시', '경상북도', '강원도', '경상남도', '전라북도', '충청북도', '대전광역시', '제주특별자치도',
       '상남도', '세종특별자치시'], dtype=object)

In [13]:
# 주소1, 주소2, 주소3 을 합쳐서 주소를 다시 입력
dat['주소'] = dat[ ['주소1', '주소2', '주소3'] ].apply(' '.join, axis=1)
dat

,지점명,주소,주소1,주소2,주소3
640,망우,서울특별시 중랑구 봉우재로 239 천우빌딩,서울특별시,중랑구,봉우재로 239 천우빌딩
641,울산세이브존,울산광역시 남구 번영로250번길 9,울산광역시,남구,번영로250번길 9
642,NC강서,서울특별시 강서구 강서로56길 17,서울특별시,강서구,강서로56길 17
643,영광,전라남도 영광군 영광읍 신남로 185-2,전라남도,영광군,영광읍 신남로 185-2
644,광주동강대,광주광역시 북구 동문대로 69,광주광역시,북구,동문대로 69
...,...,...,...,...,...
954,음성,충청북도 음성군 음성읍 수정로 53,충청북도,음성군,음성읍 수정로 53
955,롯데마트서대전,대전광역시 유성구 유성대로 26-37,대전광역시,유성구,유성대로 26-37
957,제천청전,충청북도 제천시 청전대로 169,충청북도,제천시,청전대로 169
958,대전역사,대전광역시 동구 중앙로 215,대전광역시,동구,중앙로 215


In [14]:
# 전처리 된 데이터 저장
dat.to_excel('lotteria_all_store3.xlsx', index=False)
dat.to_csv('lotteria_all_store3.csv', index=False)
dat

,지점명,주소,주소1,주소2,주소3
640,망우,서울특별시 중랑구 봉우재로 239 천우빌딩,서울특별시,중랑구,봉우재로 239 천우빌딩
641,울산세이브존,울산광역시 남구 번영로250번길 9,울산광역시,남구,번영로250번길 9
642,NC강서,서울특별시 강서구 강서로56길 17,서울특별시,강서구,강서로56길 17
643,영광,전라남도 영광군 영광읍 신남로 185-2,전라남도,영광군,영광읍 신남로 185-2
644,광주동강대,광주광역시 북구 동문대로 69,광주광역시,북구,동문대로 69
...,...,...,...,...,...
954,음성,충청북도 음성군 음성읍 수정로 53,충청북도,음성군,음성읍 수정로 53
955,롯데마트서대전,대전광역시 유성구 유성대로 26-37,대전광역시,유성구,유성대로 26-37
957,제천청전,충청북도 제천시 청전대로 169,충청북도,제천시,청전대로 169
958,대전역사,대전광역시 동구 중앙로 215,대전광역시,동구,중앙로 215
